In [580]:
import pandas as pd
import numpy as np

# Column names gotten from https://archive.ics.uci.edu/ml/datasets/seeds#)
column_names = ["Area", "Perimeter", "Compactness", "Length of kernel", "Width of kernel", "Asymmetry coefficient", "Length of kernel groove", "Type"]

df = pd.read_csv("seeds_dataset.csv", header=1, names=column_names)

df.head()

,Area,Perimeter,Compactness,Length of kernel,Width of kernel,Asymmetry coefficient,Length of kernel groove,Type
0,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
1,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
2,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
3,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1
4,14.69,14.49,0.8799,5.563,3.259,3.586,5.219,1


In [581]:
df["Type"].unique()

array([1, 2, 3], dtype=int64)

In [582]:
dict_to_replace = {1:np.int(0), 2:np.int(1), 3:np.int(2)}
y = df["Type"]
y = y.replace(dict_to_replace)
print(y[:5])

X = df.drop("Type", axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
X = scaler.fit_transform(X)
X = pd.DataFrame(data=X)
X.describe()

0    0
1    0
2    0
3    0
4    0
Name: Type, dtype: int64


,0,1,2,3,4,5,6
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.401830,0.443778,0.570327,0.410611,0.447605,0.384234,0.439266
std,0.276070,0.271096,0.215362,0.250603,0.270478,0.194463,0.242573
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.156752,0.214360,0.440336,0.202703,0.220777,0.242059,0.258986
50%,0.353636,0.387397,0.593013,0.348536,0.428724,0.369840,0.348104
75%,0.636449,0.687500,0.724365,0.611205,0.664469,0.522813,0.668882
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [583]:
data = pd.concat((X, y),axis=1)
data = data.to_numpy()


print(data)

[[0.34938621 0.34710744 0.87931034 ... 0.25145302 0.1506647  0.        ]
 [0.3068933  0.3161157  0.79310345 ... 0.19424255 0.14081733 0.        ]
 [0.52407932 0.53305785 0.86479129 ... 0.07670104 0.3229936  0.        ]
 ...
 [0.24645892 0.25826446 0.7277677  ... 0.98166664 0.26440177 2.        ]
 [0.11803588 0.16528926 0.39927405 ... 0.36834441 0.25849335 2.        ]
 [0.16147309 0.19214876 0.54718693 ... 0.63346292 0.26784835 2.        ]]


In [584]:
from sklearn.model_selection import train_test_split as tts

train, test = tts(data, train_size=0.8, random_state=2)


print(type(train))
print(type(test))
print(train.shape)
print(test.shape)

train = train.tolist()
test = test.tolist()

for row in train:
    row[-1] = int(row[-1])

for row in test:
    row[-1] = int(row[-1])

print(test[0:3])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(166, 8)
(42, 8)
[[0.07082152974504252, 0.09504132231404938, 0.4673321234119783, 0.08671171171171155, 0.15609408410548808, 0.3357084346435398, 0.23830625307730147, 2], [0.14353163361661947, 0.17768595041322266, 0.5063520871143368, 0.18975225225225278, 0.24590163934426257, 0.4377771132117178, 0.2427375677006398, 2], [0.47025495750708224, 0.5661157024793386, 0.40471869328493604, 0.5748873873873874, 0.4283677833214543, 0.24378161203500243, 0.6696208764155585, 1]]


In [585]:
from random import random, seed
import json

def print_json_dump(dump):
    print(json.dumps(dump, sort_keys=True, indent=4))

# generate weights and bias
def create_network(n_inputs:int, n_hidden_layers:int, n_neurons_for_layer:list, n_outputs:int):
    """Creates a neural network with layers, neurons with weights and bias, output neurons with weights and bias

    Args:
        n_inputs (int): The amount of input features
        n_hidden_layers (int): The amount of hidden layers desired
        n_neurons_for_layer (list): A list containing the number of neurons per hidden layer
        n_outputs (int): Amount of output neurons wanted

    Returns:
        (list): Your neural network
    """

    assert len(n_neurons_for_layer) == n_hidden_layers, \
        ("The length of this list needs to be the same as n_hidden_layers")

    network = []
    current_layer = -1

    for hidden_layer in range(n_hidden_layers):
        current_layer += 1
        layer = []
        for nodes in range(n_neurons_for_layer[current_layer]):
            if current_layer == 0:
                weights = [random() for i in range(n_inputs)]
            elif current_layer > 0:
                weights = [random() for i in range(n_neurons_for_layer[current_layer-1])]

            bias = random()
            node = {"weights":weights, "bias":bias}
            layer.append(node)

        network.append(layer)

    
    n_output_weights = len(network[-1])
    layer = []
    for i in range(n_outputs):
        weights = [random() for k in range(n_output_weights)]
        bias = random()
        node = {"weights":weights, "bias":bias}
        layer.append(node)

    network.append(layer)

    return network

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network


seed(1)
my_net = create_network(3, 3, [2, 5, 3], 3)
# my_net = initialize_network(7, 10, 3)
# print(my_net)
print(print_json_dump(my_net))

[
    [
        {
            "bias": 0.2550690257394217,
            "weights": [
                0.13436424411240122,
                0.8474337369372327,
                0.763774618976614
            ]
        },
        {
            "bias": 0.7887233511355132,
            "weights": [
                0.49543508709194095,
                0.4494910647887381,
                0.651592972722763
            ]
        }
    ],
    [
        {
            "bias": 0.8357651039198697,
            "weights": [
                0.0938595867742349,
                0.02834747652200631
            ]
        },
        {
            "bias": 0.0021060533511106927,
            "weights": [
                0.43276706790505337,
                0.762280082457942
            ]
        },
        {
            "bias": 0.22876222127045265,
            "weights": [
                0.4453871940548014,
                0.7215400323407826
            ]
        },
        {
            "bias": 0.0305899830335535

In [586]:
import numpy as np

# Neuron activation using sigmoid function
def sigmoid(x):
    val = 1/(1+np.exp(-x)) #maybe replace with math.exp
    return val

# Calculate the derivative of a neuron output
def sigmoid_deriv(x):
    return sigmoid(x)*(1-sigmoid(x))

# Calculate neuron activation for in input
def activate(weights, bias, inputs):
    activation = bias
    for i in range(len(weights)):
        activation += weights[i] * inputs[i]
    return activation

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron["weights"], neuron["bias"], inputs)
            neuron["output"] = sigmoid(activation)
            new_inputs.append(neuron["output"])
        inputs = new_inputs
    return inputs

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron["output"] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron["weights"][j] += l_rate * neuron["delta"] * inputs[j]
            neuron["bias"] += l_rate * neuron["delta"]

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print(f'>epoch={epoch}, lrate={l_rate}, error={sum_error}')

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Calculate accuracy
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Backpropagation Algorithm with stochastic gradient descent
def back_propagation(train, l_rate, n_epoch, n_hidden_layers, n_neurons_per_layer):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = create_network(n_inputs, n_hidden_layers, n_neurons_per_layer, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    return(network)

In [591]:
l_rate = 0.1
n_epoch = 10000
n_hidden_layers = 2
n_neurons_per_layer = [10, 10]

# print(train[:5])

# Train model
Model = back_propagation(train, l_rate, n_epoch, n_hidden_layers, n_neurons_per_layer)


# Make predictions on the test set
PredClass = list()
ActualClass = list()
for row in test:
    prediction = predict(Model, row)
    PredClass.append(prediction)
    ActualClass.append(row[-1])
    print('Expected=%d, Got=%d' % (row[-1], prediction))

accuracy = accuracy_metric(ActualClass, PredClass)
print("Accuracy:", accuracy)


>epoch=596, lrate=0.1, error=37.19399956605829
>epoch=597, lrate=0.1, error=37.085480309979474
>epoch=598, lrate=0.1, error=36.977374972130406
>epoch=599, lrate=0.1, error=36.86970158708603
>epoch=600, lrate=0.1, error=36.762477687504784
>epoch=601, lrate=0.1, error=36.65572029211122
>epoch=602, lrate=0.1, error=36.549445894839685
>epoch=603, lrate=0.1, error=36.443670455162525
>epoch=604, lrate=0.1, error=36.33840938962085
>epoch=605, lrate=0.1, error=36.23367756456924
>epoch=606, lrate=0.1, error=36.12948929014019
>epoch=607, lrate=0.1, error=36.02585831542656
>epoch=608, lrate=0.1, error=35.92279782487621
>epoch=609, lrate=0.1, error=35.82032043588457
>epoch=610, lrate=0.1, error=35.71843819756587
>epoch=611, lrate=0.1, error=35.617162590679655
>epoch=612, lrate=0.1, error=35.516504528680805
>epoch=613, lrate=0.1, error=35.41647435985854
>epoch=614, lrate=0.1, error=35.31708187052506
>epoch=615, lrate=0.1, error=35.218336289209276
>epoch=616, lrate=0.1, error=35.12024629180854
>epo